# Graph Neural Networks
## What are Graph Neural Networks (GNNs)?

In [24]:
#import the basics
import os
import torch
import torch_geometric
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [25]:
torch_geometric.__version__

'2.6.1'

In [26]:
# Let's verify what device we are working with
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

You are using device: cuda


Graph Neural Networks are a type of "geometric deep learning" models that use pairwise message passing. They typically have an architecture consisting of 3 types of layers. From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network):
1. Permutation equivariant: a permutation equivariant layer maps a representation of a graph into an updated representation of the same graph. In the literature, permutation equivariant layers are implemented via **pairwise message passing between graph nodes**. Intuitively, in a message passing layer, nodes update their representations by aggregating the messages received from their immediate neighbours. As such, each message passing layer increases the receptive field of the GNN by one hop.
2. Local pooling: a local pooling layer coarsens the graph via downsampling. Local pooling is used to increase the receptive field of a GNN, in a similar fashion to pooling layers in convolutional neural networks. Examples include k-nearest neighbours pooling, top-k pooling, and self-attention pooling.
3. Global pooling: a global pooling layer, also known as readout layer, provides fixed-size representation of the whole graph. The global pooling layer must be permutation invariant, such that permutations in the ordering of graph nodes and edges do not alter the final output. Examples include element-wise sum, mean or maximum.

## Attributes
- [T]he preprocessing step first
“squashes” the graph structured data into a vector of reals and
then deals with the preprocessed data using a list-based data
processing technique. However, important information, e.g., the
topological dependency of information on each node may be
lost during the preprocessing stage and the final result may depend, in an unpredictable manner, on the details of the preprocessing algorith [1] **GNNS preserve the structure of the graph it is based on.**
- It will be shown that the GNN
is an extension of both recursive neural networks and random
walk models and that it retains their characteristics. The model
extends recursive neural networks since it can process a more
general class of graphs including cyclic, directed, and undirected graphs, and it can deal with node-focused applications
without any preprocessing steps. The approach extends random
walk theory by the introduction of a learning algorithm and by
enlarging the class of processes that can be modeled. [1]
- Weights are shared across layer structures

### What is message passing?
From [wikipedia](https://en.wikipedia.org/wiki/Graph_neural_network#Message_passing_layers):
<br>
![img](./img/notebook/messagePassing.png)

## Computation Graph
"The neighbour of a node defines its computation graph" - @12:34 https://www.youtube.com/watch?v=JtDgmmQ60x8&ab_channel=AntonioLonga



# Data
Heterogeneous graphs are perfect for recommendation systems. Let's examine a data set from pytorch geometric to understand some basics about the data.

### Datasets:
"AmazonBook" - A subset of the AmazonBook rating dataset from the "LightGCN: Simplifying and Powering Graph Convolution Network for Recommendation" paper.

"MovieLens" - A heterogeneous rating dataset, assembled by GroupLens Research from the MovieLens website, consisting of nodes of type "movie" and "user". User ratings for movies are available as ground truth labels for the edges between the users and the movies ("user", "rates", "movie").

In [27]:
from torch_geometric.datasets import AmazonBook, MovieLens
from torch_geometric.transforms import Compose, ToDevice, ToUndirected

transform = Compose([ToDevice(device)])

# amazon_dataset = AmazonBook(root="./data/AmazonBook")
movielens_dataset = MovieLens(root="./data/MovieLens", transform=transform, model_name='all-MiniLM-L6-v2')

print(f"Dataset: {movielens_dataset}")
print(f"Number of graphs in dataset: {len(movielens_dataset)}")
print(f"Number of features of dataset: {movielens_dataset.num_features}")

Dataset: MovieLens()
Number of graphs in dataset: 1
Number of features of dataset: {'movie': 404, 'user': 0}


In [28]:
data = movielens_dataset[0]
print(f"data = Dataset[0]: {data}")

print(f"Number of node features of data: {data.num_features}")
print(f"Number of edge features of data: {data.num_edge_features}")
print(f"Number of nodes of data: {data.num_nodes}")
print(f"Number of edges of data: {data.num_edges}")
print(f"data is directed?: {data.is_directed()}")
print(f"data has isolated nodes?: {data.has_isolated_nodes()}")
print(f"data contains self loops?: {data.has_self_loops()}")
print(f"data node types: {data.node_types}")
print(f"data edge types: {data.edge_types}")
print(f"data is on device: {'CUDA' if data.is_cuda else 'CPU'}")

data = Dataset[0]: HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 100836],
    edge_label=[100836],
    time=[100836],
  }
)
Number of node features of data: {'movie': 404, 'user': 0}
Number of edge features of data: {('user', 'rates', 'movie'): 0}
Number of nodes of data: 10352
Number of edges of data: 100836
data is directed?: True
data has isolated nodes?: True
data contains self loops?: False
data node types: ['movie', 'user']
data edge types: [('user', 'rates', 'movie')]
data is on device: CUDA


In [29]:
data.validate(raise_on_error=True)

True

In [30]:
# print(f"Number of training edges: {data['user','rates','book'].edge_index.shape[1]}")
# print(f"Number of testing edges: {data['user','rates','book'].edge_label_index.shape[1]}")

In [31]:
# print(f"{torch.min(data['user', 'rates', 'book'].edge_index[0])}")
# print(f"{torch.max(data['user', 'rates', 'book'].edge_index[0])}")

Because there are no features on the edges or the nodes, the connections (edges) themselves are what we are training on.

- Link Prediction on MovieLens.ipynb - https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=JMGYv83WzSRr
- [1] Neural Graph Collaborative Filtering - https://dl.acm.org/doi/pdf/10.1145/3331184.3331267

In [32]:
torch_geometric.seed_everything(1234)

## Naive Graph Neural Network



In [33]:
from torch_geometric.nn import MLP, MessagePassing
from torch.nn import functional as F

"""
    GNN layer consists of:
        1.
        2.
        3.
"""

class GNNLayer(MessagePassing):
    def __init__(self, in_channels, out_channels, hidden_channels=128, aggr='add'):
        super(GNNLayer, self).__init__(aggr=aggr)
        self.message_function = MLP(in_channels= in_channels,
                                    hidden_channels=hidden_channels,
                                    out_channels=hidden_channels, num_layers=2) # default activation is ReLU
        self.update_function = MLP(in_channels=hidden_channels,
                                   hidden_channels=hidden_channels,
                                   out_channels=out_channels, num_layers=2) # default activation is ReLU

    def forward(self, x, edge_index):
        # x             [N, in_channels]
        # edge_index    [2, E]
        return self.propagate(edge_index=edge_index, x=x)

    def message(self, x_i, x_j):
        # (i,j)         -> source to target
        # x_j           [N, num_of_features]
        return self.message_function(torch.concat([x_i, x_j], dim=1))

    def update(self, aggr_out):
        return self.update_function(aggr_out)

class GNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=128, aggr='add'):
        super(GNN, self).__init__()
        self.gnn_layer_1 = GNNLayer(in_channels=in_channels, out_channels=out_channels,
                                    hidden_channels=hidden_channels, aggr=aggr)

    def forward(self, x, edge_index):
        out = self.gnn_layer_1(x, edge_index)
        return out


In [34]:
from torch_geometric.transforms import RandomLinkSplit

# lets split the data into training, validation, and test sets
transform = RandomLinkSplit(
    is_undirected=False,
    edge_types=('user', 'rates', 'movie'),
)

train_data, val_data, test_data = transform(data)

In [35]:
train_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 70586],
    time=[70586],
    edge_label=[141172],
    edge_label_index=[2, 141172],
  }
)

In [36]:
val_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 70586],
    time=[70586],
    edge_label=[20166],
    edge_label_index=[2, 20166],
  }
)

In [37]:
test_data

HeteroData(
  movie={ x=[9742, 404] },
  user={ num_nodes=610 },
  (user, rates, movie)={
    edge_index=[2, 80669],
    time=[80669],
    edge_label=[40334],
    edge_label_index=[2, 40334],
  }
)

In [38]:
from torch_geometric.loader import LinkNeighborLoader, NeighborLoader, NeighborSampler

# Define a mini-batch loader
num_samples = [30]
num_iterations = 2
batch_size = 128

loader = LinkNeighborLoader(
    train_data,
    num_neighbors= num_samples * num_iterations,
    batch_size=batch_size,
    edge_label_index=train_data.edge_types[0],
    edge_label=train_data[train_data.edge_types[0]]['edge_label']
)

# loader = NeighborLoader(
#     train_data,
#     num_neighbors=num_samples * num_iterations,
#     batch_size=batch_size
# )

In [39]:
sampled_data = next(iter(loader))
print(sampled_data)

HeteroData(
  movie={
    x=[124, 404],
    n_id=[124],
    num_sampled_nodes=[3],
  },
  user={
    num_nodes=463,
    n_id=[463],
    num_sampled_nodes=[3],
  },
  (user, rates, movie)={
    edge_index=[2, 2345],
    time=[2345],
    edge_label=[128],
    edge_label_index=[2, 128],
    e_id=[2345],
    num_sampled_edges=[2],
    input_id=[128],
  }
)


In [40]:
len(loader)

552

In [ ]:
num_of_epochs = 100
model = GNN(in_channels=610, out_channels=9742)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(num_of_epochs):
    for batch in loader:
        target = batch['user', 'rates', 'movie'].edge_label
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index)